# TO GATHER LINKS FROM CAR DEKHO

<b> STEP 1: </b> Setting up chromium browser for colab

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

__STEP 2:__ Importing all the required packages

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.chrome.options import Options

In [ ]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
def get_car_divs(driver):
    driver.maximize_window()
    driver.get('https://www.cardekho.com/used-cars+in+hyderabad')
    time.sleep(2)

    for i in range(8000):
        driver.execute_script("scrollBy(0,200)")
        time.sleep(1)

    car_divs = driver.find_elements(By.CLASS_NAME, 'NewUcExCard')
    return car_divs

In [ ]:
def car_links(car_divs):
    links = []

    for div in car_divs:
        link = div.find_element(By.TAG_NAME, 'a')
        links.append(link.get_attribute('href'))
        
    return links

In [ ]:
def get_car_links():
    links = car_links(car_divs)
    return links

__STEP 3:__ Gathering all the links from the main function

In [ ]:
if __name__ == '__main__':
    driver = get_driver()
    car_divs = get_car_divs(driver)

    links = get_car_links()

    driver.close()

In [ ]:
len(links)

__STEP 4:__ Saving all the links in Google Drive in the form of "links.txt"

In [ ]:
from google.colab import drive
drive.mount("mnt")

In [ ]:
%cd "/content/mnt/MyDrive/Scrapper"

In [ ]:
with open('hyderabad_links.txt','w') as f:
    for link in links:
        f.write(link + '\n')